# Import necessary Modules

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import pickle
import numpy as np
import matplotlib.pyplot as plt


# A function to read and load the pickle file

In [ ]:

def load_pickle(path):
    with open(path,'rb') as f:
        return pickle.load(f)

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


train_ds = load_pickle('/kaggle/input/traffic-signs-preprocessed/train.pickle')
test_ds = load_pickle('/kaggle/input/traffic-signs-preprocessed/test.pickle')


print(type(train_ds))
print(train_ds.keys())
print(train_ds["features"].shape, train_ds["labels"].shape)

# Custom Module for formatting the data for using it in the dataloader

In [ ]:
class TrafficDs(Dataset):
    def __init__(self,features,labels,transform=None):
        self.features = features
        self.labels = labels
        self.transform = transform
    
        
    def __len__(self):
        return len (self.features)


    def __getitem__(self, idx):
        image = self.features[idx]  
        label = self.labels[idx]
    
        # Fix shape
        import numpy as np
        image = np.squeeze(image)  # (32, 32, 3)
    
        from PIL import Image
        image = Image.fromarray(image)
    
        if self.transform:
            image = self.transform(image)
    
        return image, label        

# Creating the datasets

In [ ]:
train_dataset = TrafficDs(
    train_ds["features"], train_ds["labels"], transform=transform
)
test_dataset = TrafficDs(
    test_ds["features"], test_ds["labels"], transform=transform
)


T_ind = int(0.75*len(train_dataset))
V_ind = int(int(len(train_dataset)) -  T_ind)
train , val =  random_split(train_dataset,[T_ind,V_ind])

train_loader = DataLoader(train, batch_size=64, shuffle=True)
val_loader = DataLoader( val, batch_size=64, shuffle=False)
test_loader = DataLoader( test_dataset, batch_size=64, shuffle=False)


# see the image and label shape

In [ ]:
images, labels = next(iter(train_loader))
print(images.shape, labels.shape)

# Visulaise the image in the files

In [ ]:
img = images[10]
img = img * 0.5 + 0.5
img = img.permute(1, 2, 0).cpu().numpy()
plt.imshow(img)
plt.axis('off')  
plt.show()


# Custom CNN Network for calssification

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class TrafficNet(nn.Module):
    def __init__(self):
        super(TrafficNet, self).__init__()
        self.cn1 = nn.Conv2d(3,32,3,1)
        self.cn2 = nn.Conv2d(32,64,3,1)
        self.fc1 =  nn.Linear(64*6*6,128)
        self.fc2 = nn.Linear(128,43)

    
    def forward(self,x):
        x = F.relu(self.cn1(x))
        x = F.max_pool2d(x,2)
        
        x = F.relu(self.cn2(x))
        x = F.max_pool2d(x,2)

        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x= self.fc2(x)
        return x
        
        


# Training and Validating the Model

In [ ]:
model = TrafficNet()
losses = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr = 0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

num_epochs = 15
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training
    model.train()
    tot_loss = 0
    for img, lab in train_loader:
        img, lab = img.to(device), lab.to(device)
        opt.zero_grad()
        out = model(img)
        loss = losses(out, lab)
        loss.backward()
        opt.step()
        tot_loss += loss.item()
    avg_train_loss = tot_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for img, lab in val_loader:
            img, lab = img.to(device), lab.to(device)
            out = model(img)
            loss = losses(out, lab)
            val_loss += loss.item()
            
    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")



# Plotting the losses

In [ ]:
plt.plot(range(1, 16), train_losses, marker='o')
plt.plot(range(1, 16), val_losses, marker='x')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss vs Epoch")
plt.grid(True)
plt.show()

# zooming in the graph

In [ ]:
plt.plot(range(4, 16), train_losses[3:], marker='o')
plt.plot(range(4, 16), val_losses[3:], marker='x')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss vs Epoch")
plt.grid(True)
plt.show()